In [8]:
import cv2 ### opencv

In [9]:
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

In [10]:
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
    return faces

In [13]:
## capturing the screen 

import numpy as np
from mss import mss


with mss() as sct:
    # Choose the monitor you want to capture. For example, use monitor[1] for the first monitor.
    monitor = sct.monitors[2]

    while True:
        # Grab the screen image
        img = np.array(sct.grab(monitor))
        # Convert the image from BGRA to BGR
        screen_frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        # Pass the screen_frame to your face detection function
        faces = detect_bounding_box(screen_frame)

        # Optionally, if detect_bounding_box doesn't draw on the frame,
        # you could draw rectangles here based on the returned faces.
        # For example:
        # for (x, y, w, h) in faces:
        #     cv2.rectangle(screen_frame, (x, y), (x+w, y+h), (0, 255, 0), 4)

        cv2.imshow("My Face Detection Project", screen_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cv2.destroyAllWindows()

In [11]:
from deepface import DeepFace
import os

In [14]:
# Load Haar cascade for face detection.
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Optionally, precompute your reference embeddings.
# For example, you might have a folder "reference_faces" with subfolders or images named with the label.
# You can run this once at startup to build your embeddings database.
# Alternatively, DeepFace.find() can search a folder directly.
db_path = r"C:\Users\User\Pictures\Face_recog_db"
def detect_and_recognize(screen_frame):
    # Convert frame to grayscale for face detection.
    gray_image = cv2.cvtColor(screen_frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    
    for (x, y, w, h) in faces:
        # Extract face region of interest (ROI)
        face_roi = screen_frame[y:y+h, x:x+w]
        # Convert ROI from BGR to RGB (DeepFace expects RGB images)
        face_roi_rgb = face_roi ##cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        
        try:
            # Use DeepFace.find() to compare the detected face against your reference database.
            # Setting enforce_detection=False to avoid errors if a face isn't perfectly aligned.
            
            
            results = DeepFace.find(img_path=face_roi_rgb, db_path=db_path, model_name= "Facenet", enforce_detection=False)

            ##result_ver = DeepFace.verify(face_roi_rgb, db_path, model_name="VGG-Face")

            ##Label = result_ver["verified"]


            
            
            if len(results) > 0 and len(results[0]) > 0:
                # Assuming results is a list of DataFrames (one per model)
                # Get the first match's identity, then extract the name from the file path.
                identity_path = results[0].iloc[0]["identity"]
                label = identity_path.split("/")[-1].split("\\")[-1]  # works for Linux/Mac and Windows paths
            else:
                label = "Unknown"
        except Exception as e:
            # If DeepFace fails to process, label as Unknown.
            label = "Unknown_from_exception"
            print("Recognition error:", e)
        
        # Draw rectangle and label on the frame.
        cv2.rectangle(screen_frame, (x, y), (x+w, y+h), (0, 255, 0), 4)
        cv2.putText(screen_frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return screen_frame

# Capture the screen feed from the second monitor.
with mss() as sct:
    monitor = sct.monitors[2]
    while True:
        # Grab the screen image.
        img = np.array(sct.grab(monitor))
        # Convert image from BGRA to BGR.
        screen_frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        
        # Detect faces and perform recognition.
        processed_frame = detect_and_recognize(screen_frame)
        
        cv2.imshow("My Face Detection & Recognition Project", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cv2.destroyAllWindows()

25-03-04 11:56:10 - Searching [[[121  69  39]
  [118  66  35]
  [135  83  53]
  ...
  [247 227 190]
  [245 225 188]
  [246 223 188]]

 [[134  82  53]
  [136  84  54]
  [137  85  55]
  ...
  [246 227 190]
  [244 225 188]
  [246 223 188]]

 [[144  93  62]
  [145  93  63]
  [125  73  43]
  ...
  [247 227 190]
  [245 224 187]
  [246 223 188]]

 ...

 [[146 101  65]
  [126  81  50]
  [117  73  43]
  ...
  [ 49  60  98]
  [ 46  51  78]
  [ 49  46  65]]

 [[137 101  75]
  [123  91  69]
  [113  81  63]
  ...
  [ 43  59  95]
  [ 40  49  75]
  [ 45  43  62]]

 [[ 99  79  69]
  [ 82  66  60]
  [ 70  54  54]
  ...
  [ 40  57  91]
  [ 35  44  68]
  [ 37  37  55]]] in 6 length datastore
25-03-04 11:56:21 - find function duration 11.715944528579712 seconds
25-03-04 11:56:21 - Searching [[[150 175 191]
  [151 175 191]
  [151 175 191]
  ...
  [161 180 193]
  [161 180 193]
  [161 180 193]]

 [[149 175 191]
  [150 175 191]
  [150 175 191]
  ...
  [162 181 194]
  [162 181 194]
  [162 181 194]]

 [[148 174

In [24]:
from deepface import DeepFace
import os
result = DeepFace.verify(r"C:\Users\User\Documents\Artificial Intelligence Durham College\Semester 2\CAPSTONE II\aidi_winter_2025_capstone_untitled_team_automatic_attendance_system\model_tests\andres_open_cv_model\refference_images\Joshua_Yap.jpeg",
                 r"C:\Users\User\Documents\Artificial Intelligence Durham College\Semester 2\CAPSTONE II\aidi_winter_2025_capstone_untitled_team_automatic_attendance_system\model_tests\andres_open_cv_model\refference_images\Majid_Abdullah.jpeg", model_name="Facenet")


result["verified"]

False

In [50]:


results = DeepFace.find(img_path= r"C:\Users\User\Pictures\test_image.png",
                        db_path= r"C:\Users\User\Pictures\Face_recog_db",
                        model_name="VGG-Face",
                         enforce_detection= False);

25-02-21 19:43:29 - Found 3 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 3/3 [00:04<00:00,  1.37s/it]


25-02-21 19:43:33 - There are now 3 representations in ds_model_vggface_detector_opencv_aligned_normalization_base_expand_0.pkl
25-02-21 19:43:33 - Searching C:\Users\User\Pictures\test_image.png in 3 length datastore
25-02-21 19:43:35 - find function duration 5.639761924743652 seconds


In [52]:
results[1]


,identity,hash,target_x,target_y,target_w,target_h,source_x,source_y,source_w,source_h,threshold,distance
0,C:\Users\User\Pictures\Face_recog_db\Joshua_Ya...,97c7374abaf4deae93ee83a4090065fad2c1de1f,213,281,820,820,0,38,120,120,0.68,0.044699


In [15]:
sct.grab?

Signature: sct.grab(monitor: 'Monitor | tuple[int, int, int, int]', /) -> 'ScreenShot'
Docstring:
Retrieve screen pixels for a given monitor.

Note: *monitor* can be a tuple like the one PIL.Image.grab() accepts.

:param monitor: The coordinates and size of the box to capture.
                See :meth:`monitors <monitors>` for object details.
:return :class:`ScreenShot <ScreenShot>`.
File:      c:\users\user\appdata\local\programs\python\python311\lib\site-packages\mss\base.py
Type:      method

In [8]:
cv2.destroyAllWindows()

In [12]:
with mss() as sct:
    while True:
        monitor = sct.monitors[2]
        img = np.array(sct.grab(monitor))
            # Convert image from BGRA to BGR.
        screen_frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        cv2.imshow("title",screen_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cv2.destroyAllWindows()

In [28]:
import os

db_path = r"C:\Users\User\Documents\Artificial Intelligence Durham College\Semester 2\CAPSTONE II\aidi_winter_2025_capstone_untitled_team_automatic_attendance_system\model_tests\andres_open_cv_model\refference_images"

if os.path.exists(db_path):
    contents = os.listdir(db_path)
    print("Contents of the directory:", contents)
else:
    print("Directory does not exist!")


Contents of the directory: ['August_Bergman.png', 'Herify.jfif', 'Joshua_Yap.jpeg', 'luisa_f.png', 'Majid_Abdullah.jpeg']


In [25]:
DeepFace.find?

Signature:
DeepFace.find(
    img_path: Union[str, numpy.ndarray],
    db_path: str,
    model_name: str = 'VGG-Face',
    distance_metric: str = 'cosine',
    enforce_detection: bool = True,
    detector_backend: str = 'opencv',
    align: bool = True,
    expand_percentage: int = 0,
    threshold: Optional[float] = None,
    normalization: str = 'base',
    silent: bool = False,
    refresh_database: bool = True,
    anti_spoofing: bool = False,
) -> List[pandas.core.frame.DataFrame]
Docstring:
Identify individuals in a database
Args:
    img_path (str or np.ndarray): The exact path to the image, a numpy array in BGR format,
        or a base64 encoded image. If the source image contains multiple faces, the result will
        include information for each detected face.

    db_path (string): Path to the folder containing image files. All detected faces
        in the database will be considered in the decision-making process.

    model_name (str): Model for face recognition. Option

In [7]:
cv2.cvtColor?


Docstring:
cvtColor(src, code[, dst[, dstCn[, hint]]]) -> dst
.   @brief Converts an image from one color space to another.
.   
.   The function converts an input image from one color space to another. In case of a transformation
.   to-from RGB color space, the order of the channels should be specified explicitly (RGB or BGR). Note
.   that the default color format in OpenCV is often referred to as RGB but it is actually BGR (the
.   bytes are reversed). So the first byte in a standard (24-bit) color image will be an 8-bit Blue
.   component, the second byte will be Green, and the third byte will be Red. The fourth, fifth, and
.   sixth bytes would then be the second pixel (Blue, then Green, then Red), and so on.
.   
.   The conventional ranges for R, G, and B channel values are:
.   -   0 to 255 for CV_8U images
.   -   0 to 65535 for CV_16U images
.   -   0 to 1 for CV_32F images
.   
.   In case of linear transformations, the range does not matter. But in case of a non-linear
.  